## Create relations between items in SSHOMP

Idea: Read all descriptions and find hidden relations

Prep: Imports

In [1]:
import pandas as pd #to manage dataframes
import json #to manage json objects
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel
import requests
#from tqdm import tqdm, tqdm_notebook #for the progress bar
import time
from IPython.display import JSON
from collections import defaultdict

## 1. Get all Items

In [2]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...


In [3]:
df_all_items=pd.concat([df_tool_flat, df_publication_flat, df_trainingmaterials_flat, df_workflows_flat, df_datasets_flat]) #merge into one dataframe
# Filter the dataframe to keep only items with status == 'approved'
df_all_items_approved = df_all_items[df_all_items['status'] == 'approved']

In [4]:
#df_all_items.transpose()

## 2. Read all descriptions and find mentions of other tools here

In [5]:
#df_all_items = pd.read_pickle('relations') #shortcut for debugging

In [6]:
mentions = defaultdict(list)
def find_mentions(df_all_items):
    #tqdm.pandas()  # Use this if you want to use tqdm for progress visualization
#     mentions = defaultdict(list)

    # Iterate over each row in the DataFrame
#     for index, row in tqdm(df_all_items.iterrows(), total=len(df_all_items)):
        
    for index, row in df_all_items.iterrows():
        tool = row['label']
        description = row['description']
        pers_id = row['persistentId']
        category = row['category']
        # Iterate over each tool to check if it is mentioned in the description
        for mentioned_tool, mentioned_pers_id, mentioned_category in zip(df_all_items['label'], df_all_items['persistentId'], df_all_items['category']):
            if mentioned_tool != tool and f" {mentioned_tool} " in f" {description} ":
                #print (mentioned_tool)
                mentions[tool].append([mentioned_pers_id, mentioned_category])
                #print(f'Item {tool} mentioned in {mentioned_tool}')  # Print statement for debugging

    # Add a new column 'mentioned_in' to store PersID and category of mentioned tools
    #df_all_items['mentioned_in'] = df_all_items['persistentId'].apply(lambda x: mentions[df_all_items.loc[df_all_items['persistentId'] == x, 'label'].values[0]])
    


In [7]:
find_mentions(df_all_items)

In [8]:
df_all_items['mentioned_in'] = df_all_items['persistentId'].apply(lambda x: mentions[df_all_items.loc[df_all_items['persistentId'] == x, 'label'].values[0]])

In [9]:
df_filtered = df_all_items[df_all_items['mentioned_in'].apply(len) > 0]

In [10]:
#df_filtered.to_pickle('relations') #shortcut for debugging

In [11]:
utils=hel.Util();
my_df=df_filtered[['category','persistentId', 'label','status', 'mentioned_in']].copy()
my_df['checkIt']=my_df.mentioned_in.apply(lambda y:f'{y[0][1]}/{y[0][0]}')
clickable_nv_table = my_df.style.format({'checkIt': utils.make_clickable})
clickable_nv_table

,category,persistentId,label,status,mentioned_in,checkIt
2,tool-or-service,OgfTLA,3DHOP: 3D Heritage Online Presenter,approved,"[['x9cgZf', 'tool-or-service']]",tool-or-service/x9cgZf
7,tool-or-service,rdwzoM,4th Dimension,approved,"[['4iYxo7', 'tool-or-service']]",tool-or-service/4iYxo7
19,tool-or-service,j8MRN1,Active Server Pages (ASP),approved,"[['x9cgZf', 'tool-or-service']]",tool-or-service/x9cgZf
25,tool-or-service,6ssOMC,Adobe Dreamweaver,approved,"[['x9cgZf', 'tool-or-service']]",tool-or-service/x9cgZf
27,tool-or-service,aeQ2f6,Adobe InDesign,approved,"[['Px3QPk', 'tool-or-service']]",tool-or-service/Px3QPk
29,tool-or-service,RY4LgT,Advene,approved,"[['98ulMr', 'tool-or-service']]",tool-or-service/98ulMr
37,tool-or-service,knCcCv,Aïoli platform,approved,"[['E7Dyt0', 'publication']]",publication/E7Dyt0
38,tool-or-service,HRUmqc,A JavaScript library for rendering music notation and guitar tablature.: 0xfe/vexflow,approved,"[['olMsqO', 'tool-or-service'], ['40W1iw', 'training-material']]",tool-or-service/olMsqO
41,tool-or-service,8MeMkv,Alpage,approved,"[['ZAUXU6', 'tool-or-service']]",tool-or-service/ZAUXU6
45,tool-or-service,tAGITn,Alveo,approved,"[['tt0gGA', 'tool-or-service']]",tool-or-service/tt0gGA


## 3. Check if these new relations make sense

In [12]:
def construct_mp_urls(server, category, persId):
    if server == 'stage':
        server = 'https://sshoc-marketplace-stage.acdh-dev.oeaw.ac.at'
    elif server == 'prod':
        server = 'https://marketplace.sshopencloud.eu'
    else:
        print ('Error, server not recognized')
        return
    url = f'{server}/{category}/{persId}'
    return url

In [13]:
df_filtered['mentioned_urls'] = df_filtered['mentioned_in'].apply(lambda x: [construct_mp_urls('prod', item[1], item[0]) for item in x])

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_83060/2867242451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['mentioned_urls'] = df_filtered['mentioned_in'].apply(lambda x: [construct_mp_urls('prod', item[1], item[0]) for item in x])


Test these new relations with curation TF and decide if we want to add

In [14]:
df_filtered.to_csv('relations.csv')

## 4. Create these new relations through the API

This needs /api/items-relations/{{ _.subjectId }}/{{ _.objectId }} and code "is mentioned in"